<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/PartisanBiasDetection/NewsRSS_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Dependancies
!pip install requests beautifulsoup4 --quiet

#Import libraries
import requests
from bs4 import BeautifulSoup
import time

In [15]:
# @title Define Utility Functions

#Article Scraper
def scrape_article_text(url):
    # Send a GET request to the URL
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find elements containing text - this part may need customization
        # For example, assuming article text is within <p> tags
        article_text = ' '.join(p.get_text() for p in soup.find_all('p'))
        return article_text
    else:
        return "Error: Unable to fetch the webpage."

#Article Scraper with User Agent
def scrape_article_text_useragent(url,ua):
    # Send a GET request to the URL
    response = requests.get(url, headers=ua)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find elements containing text - this part may need customization
        # For example, assuming article text is within <p> tags
        article_text = ' '.join(p.get_text() for p in soup.find_all('p'))
        return article_text
    else:
        return "Error: Unable to fetch the webpage."

#Query for LLM
def query(payload, headers):
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()  # Expecting this to always be a list for simplicity
    else:
        return [{"error": response.text}]

#Political Bias Classification using LLM
def reduce_and_query(initial_text, llm_instructions, reduction_fraction, headers):
    text = initial_text
    while True:
        payload = {"inputs": llm_instructions + text}
        response = query(payload, headers)
        # Since we expect a list, check the first item for an 'error' key
        if "error" in response[0]:
            error_message = response[0]["error"]
            if "token" in error_message.lower():
                # Reduce the text by the specified fraction if it's a token limit error
                new_length = int(len(text) * (1 - reduction_fraction))
                text = text[:new_length]
            else:
                # Return the error message directly if it's a different kind of error
                return error_message
        else:
            # If there's no 'error' key, assume the response is successful
            return response

In [122]:
# CNN Article Link Scraper
page = requests.get('https://www.cnn.com/politics')
soup = BeautifulSoup(page.text, 'html.parser')

links = []
for l in soup.find_all('a'):
    links.append(l.get('href'))
links = filter(None, links)
linkEnds = [i for i in links if i.startswith('/2') and "/video/" not in i]
result = ['https://www.cnn.com' + i for i in linkEnds]
links = set(result)
links = list(links)

#CNN Article Text Scraper
artTxt = []
for art in links:
  article_text = scrape_article_text(art)
  artTxt.append(article_text)

#CNN Article Header Scraper
headers = []
for h in links:
  hp = requests.get(h)
  souph = BeautifulSoup(hp.text, 'html.parser')
  span = souph.find_all('h1')
  header= str(span)
  i = header.find(">")
  j = header.find("</h1>")
  x = header[i+8:j]
  y = x.split('\n', 1)[0]
  headers.append(y)

#CNN News Article PubDate Scraper
  #Try to nest this within the above loop

In [115]:
# @title Fox News [completed: links, articleTxt, headers, and pubdate]
#Fox News Article Text Link Scraper
url = 'https://moxie.foxnews.com/google-publisher/politics.xml'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
artTxt = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  links.append(y)
  article_text = scrape_article_text(y)
  artTxt.append(article_text)

#Fox News Article Header Scraper
headers = []
for item in soup.find_all("item"):
  header= str(item)
  i = header.find("<title>")
  j = header.find("</title>")
  x = header[i+7:j]
  y = x.split('\n', 1)[0]
  headers.append(y)
FoxNews_headers = headers

#Fox News Article PubDate Scraper
pubds = []
for item in soup.find_all("item"):
  pubd= str(item)
  i = pubd.find("<pubdate>")
  j = pubd.find("</pubdate>")
  x = pubd[i+9:j]
  y = x.split('\n', 1)[0]
  pubds.append(y)
FoxNews_pubdate = pubds

<ipython-input-115-1a2f6ef9f2db>:5: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


In [117]:
item

<item>
<link/>https://www.foxnews.com/politics/voters-have-largely-positive-response-exchange-between-vance-walz-shootings
                        <guid ispermalink="true">https://www.foxnews.com/politics/voters-have-largely-positive-response-exchange-between-vance-walz-shootings</guid>
<title>Voters have largely positive response to exchange between Vance and Walz on shootings</title>
<description>Voters from all groups had a mostly positive response to Ohio Sen. JD Vance and Minnesota Gov. Tim Walz&amp;apos;s exchange on school shootings and gun control during the debate.</description>
<content:encoded>&lt;p&gt;Voters watching Tuesday’s debate between &lt;a href="https://www.foxnews.com/category/person/jd-vance" target="_blank" rel="noopener"&gt;Ohio Sen. JD Vance&lt;/a&gt; and Minnesota Gov. Tim Walz had a largely positive response to an exchange between the two vice presidential candidates on shootings.&lt;/p&gt;&lt;p&gt;"I got a 17-year-old, and he witnessed a shooting at a commun

In [116]:
soup

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:media="http://search.yahoo.com/mrss/">
<channel>
<title>Latest Political News on Fox News</title>
<link/>https://www.foxnews.com/politics
        <description>Read all about the political news happening with Fox News. Learn about political parties, political campaigns, and international politics today.</description>
<copyright>Copyright 2024 FOX News Network</copyright>
<language>en-us</language>
<pubdate>Thu, 03 Oct 2024 16:00:42 -0400</pubdate>
<image>
<url>https://global.fncstatic.com/static/orion/styles/img/fox-news/logos/fox-news-desktop.png</url>
<title>Latest Political News on Fox News</title>
<link/>https://www.foxnews.com/politics
        </image>
<atom:link href="https://moxie.foxnews.com/google-publisher/politics.xml" rel="self" type="applicatio

In [7]:
#NYT Article Link Scraper
url = 'https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  links.append(y)
NYT_links = links

<ipython-input-7-d5bd07c2d47a>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


In [9]:
#USA Today Article Link Scraper
page = requests.get('https://www.usatoday.com/news/politics/')
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://www.usatoday.com/news/politics/"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
'''
   /politics removes livestream & corrections & world politics
   also removes other articles e.g. /investigation (ryan routh article)
'''
linkEnds = [i for i in allLinks if i.startswith('/story/news/politics')]
result = ['https://www.usatoday.com' + i for i in linkEnds]
USAToday_links = set(result)
USAToday_links = list(USAToday_links)

In [11]:
#NY Post Article Link Scraper
#check lxml thing later
headers = {"user-agent": "Mozilla/5.0"}
url = 'https://nypost.com/politics/feed/'
html_page = requests.get(url, headers=headers)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link = str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  #remove videos
  if not (y.startswith('https://nypost.com/video')):
    links.append(y)
NYPost_links = links

<ipython-input-11-4f9876e9c3ac>:6: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


In [13]:
#People Article Link Scraper
#without user-agent
page = requests.get('https://people.com/politics/')
soup = BeautifulSoup(page.text, 'html.parser')

allLinks = []
link = "https://people.com/politics/"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
#identifier for political articles seems to be a 7 digit # that starts w 87 at the end of the link
#check to confirm, at least i got astrology out of there
linkEnds = [i for i in allLinks if (len(i) > 8 and i[-7] == '8' and i[-6] == '7')]
# result = ['https://people.com' + i for i in linkEnds]
People_links = set(linkEnds)
People_links = list(People_links)

In [ ]:
# @title [Potenitally discontinuted] Ver 2: People Article Link Scraper
#with user-agent
#try if the other one breaks
'''
#Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)
#reference: https://www.stanventures.com/blog/googlebot-user-agent-string/
#Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/W.X.Y.Z Safari/537.36
headers = {"user-agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
#Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/W.X.Y.Z Safari/537.36
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35
page = requests.get('https://people.com/politics/',headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')
# print(soup)
allLinks = []
link = "https://people.com/politics/"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
#identifier for political articles seems to be a 7 digit # that starts w 87 at the end of the link
#check to confirm, at least i got astrology out of there
linkEnds = [i for i in allLinks if (len(i) > 8 and i[-7] == '8' and i[-6] == '7')]
# result = ['https://people.com' + i for i in linkEnds]
People_links = set(linkEnds)
People_links = list(People_links)
'''

In [16]:
#Daily Mail Article Link Scraper
#use this one
page = requests.get('https://www.dailymail.co.uk/news/us-politics/index.html')
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://www.dailymail.co.uk/news/us-politics/index.html"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
linkEnds = [i for i in allLinks if i.startswith('/news/article')]
result = ['https://www.dailymail.co.uk/' + i for i in linkEnds]
DailyMail_links = set(result)
DailyMail_links = list(DailyMail_links)

In [ ]:
# @title [Potenitally discontinuted] Daily Mail RSS Article Link Scraper - articles not as relevant
#don't use this one
'''
url = 'https://www.dailymail.co.uk/ushome/index.rss'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  links.append(y)
dm_links = links
list(set(DailyMail_links).intersection(dm_links))
'''

'url = \'https://www.dailymail.co.uk/ushome/index.rss\'\nhtml_page = requests.get(url)\nsoup = BeautifulSoup(html_page.text, "lxml")\nlinks = []\nfor item in soup.find_all("item"):\n  link= str(item)\n  i = link.find("<link/>")\n  j = link.find("<guid")\n  x = link[i+7:j]\n  y = x.split(\'\n\', 1)[0]\n  links.append(y)\ndm_links = links\nlist(set(DailyMail_links).intersection(dm_links))\n'

In [18]:
#Newsweek Article Link Scraper
headers = {"user-agent": "Mozilla/5.0"}
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35
page = requests.get('https://www.newsweek.com/politics',headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://www.newsweek.com/politics"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
linkEnds = [i for i in allLinks if (len(i) > 8 and i[-7] == '1')]
# linkEnds = [i for i in allLinks]
result = ['https://www.newsweek.com' + i for i in linkEnds]
Newsweek_links = set(result)
Newsweek_links = list(Newsweek_links)

# initial error
# print(soup.find_all('body'))
# [<body>
# <center><h1>403 Forbidden</h1></center>
# </body>]

# reference: https://stackoverflow.com/questions/40255128/how-to-parse-the-website-using-beautifulsoup
# reference: https://www.useragentlist.net/

In [20]:
#NBC News Article Link Scraper
url = 'https://feeds.nbcnews.com/nbcnews/public/politics'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link = str(item)
  i = link.find("<guid>")
  j = link.find("</guid")
  x = link[i+6:j]
  y = x.split('\n', 1)[0]
  if y[0:4] == 'http':
    links.append(y)
# probably unnecessary, try adding in if it breaks
# if "https://www.nbcnews.com/"  in links:
#   links.remove("https://www.nbcnews.com/")
NBCNews_links = links

In [22]:
#Forbes Article Link Scraper
page = requests.get('https://www.forbes.com/topics/politics/')
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://www.forbes.com/topics/politics/"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
#all articles contain date (e.g. /2024/09/18)
relevantLinks = [i for i in allLinks if '/2' in i]
Forbes_links = set(relevantLinks)
Forbes_links = list(Forbes_links)

In [24]:
#AP News Article Link Scraper
page = requests.get('https://apnews.com/politics')
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://apnews.com/politics"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
relevantLinks = [i for i in allLinks if '/article' in i]
APNews_links = set(relevantLinks)
APNews_links = list(APNews_links)

In [26]:
#BBC Article Link Scraper
#via U.S. Politics page
page = requests.get('https://www.bbc.com/news/topics/cwnpxwzd269t')
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://www.bbc.com/news/topics/cwnpxwzd269t"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
linkEnds = [i for i in allLinks if '/article' in i]
result = ['https://www.bbc.com' + i for i in linkEnds]
BBC_links = set(result)
BBC_links = list(BBC_links)

In [29]:
#Washington Post Article Link Scraper - in progress
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
#headers = {"user-agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
#headers = {"user-agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
#User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35

page = requests.get('https://www.washingtonpost.com/politics/',headers=headers)
# print(BS(page.content, 'lxml'))
soup = BeautifulSoup(page.text, 'html.parser')
# print(soup)

allLinks = []
link = "https://www.washingtonpost.com/politics/"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
linkEnds = [i for i in allLinks if "/politics/" and "/2" in i]
links = [i if i.startswith("https") else "https://www.washingtonpost.com/" + i for i in linkEnds]

WP_links = set(links)
WP_links = list(WP_links)
WP_links

#RSS: https://feeds.washingtonpost.com/rss/rss_the-fix?itid=lk_inline_manual_5

['https://www.washingtonpost.com/elections/interactive/2024/vice-presidential-debate-voter-poll/',
 'https://www.washingtonpost.com//politics/2024/10/03/no-amount-evidence-will-convince-republicans-trumps-2020-guilt/?itid=mr_politics_2',
 'https://www.washingtonpost.com//politics/2024/10/03/who-does-trump-plan-deport-who-trump-wants-deport/?itid=mr_politics_4',
 'https://www.washingtonpost.com/politics/2024/10/02/biden-harris-hurricane-helene/',
 'https://www.washingtonpost.com/politics/2024/10/03/who-does-trump-plan-deport-who-trump-wants-deport/',
 'https://www.washingtonpost.com/politics/2024/10/03/presidential-polls-polling-average-washington-post/',
 'https://www.washingtonpost.com/politics/2024/10/03/tina-peters-colorado-elections-prison/',
 'https://www.washingtonpost.com/politics/2024/10/03/no-amount-evidence-will-convince-republicans-trumps-2020-guilt/',
 'https://www.washingtonpost.com/politics/2024/10/03/harris-liz-cheney-wisconsin-campaign-ripon-republican/',
 'https://www.

In [30]:
#CNBC Article Link Scraper
url = 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=10000113'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  #removes white space at end of links
  y = x.split('\n', 1)[0].strip()
  links.append(y)
CNBC_links = links

<ipython-input-30-bec0c30b8cb0>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


In [32]:
#CBS News Article Link Scraper
url = 'https://www.cbsnews.com/latest/rss/politics'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  if "/news/" in y:
    links.append(y)
CBS_links = links
CBS_links

<ipython-input-32-36164bfd68a9>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


['https://www.cbsnews.com/news/former-mesa-county-clerk-tina-peters-sentenced-colorado-election-interference-case/',
 'https://www.cbsnews.com/news/supreme-court-new-term-cases-to-watch/',
 'https://www.cbsnews.com/news/kamala-harris-campaign-liz-cheney-wisconsin-republicans/',
 'https://www.cbsnews.com/news/venezuelans-legal-status-chnv-program/',
 'https://www.cbsnews.com/news/vp-debate-ratings-2024/',
 'https://www.cbsnews.com/news/us-microsoft-russia-domains-seized/',
 'https://www.cbsnews.com/news/biden-florida-georgia-hurricane-helene-damage/',
 'https://www.cbsnews.com/news/melania-trump-abortion-rights-memoir/',
 'https://www.cbsnews.com/news/new-york-jets-woody-johnson-ambassador-ethics/',
 'https://www.cbsnews.com/news/tim-walz-gus-witnessed-st-paul-community-center-shooting/',
 'https://www.cbsnews.com/news/vp-debate-replay-2024/',
 'https://www.cbsnews.com/news/oct-7-hamas-attack-one-year-mark-u-s-intelligence-warning-violent-extremism/',
 'https://www.cbsnews.com/news/tim-

In [ ]:
# @title [Potenitally discontinuted] ABC News Article Link Scraper - in progress

# headers = {"user-agent": "Mozilla/5.0"}
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35
html_page = requests.get('https://abcnews.go.com/abcnews/politicsheadlines')
#lxml-xml needed so it can parse the CDATA (where the links are) ? jk
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("CDATA")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  links.append(y)

ABC_links = links
ABC_links

In [34]:
#ABC Article Link Scraper html
page = requests.get('https://abcnews.go.com/Politics')
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://abcnews.go.com/Politics"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
links = [i for i in allLinks if "/Politics/" in i and "/video/" not in i]
ABC_links = set(links)
ABC_links = list(ABC_links)

In [ ]:
#Extract article text from links for all news sources
#Use this to text article link efficacy?

#CNBC_links includes ad text in the beginning(?)
url = CBS_links[0]
article_text = scrape_article_text(url)
print(article_text)

Watch CBS News 
Updated on:  September 18, 2024 / 10:18 PM EDT
          / CBS/AP
         Iranian hackers sought to interest President Biden's campaign in information stolen from the rival campaign of former President Donald Trump, sending unsolicited emails to people connected to the Democratic president in an effort to interfere in the 2024 election, the FBI and other federal agencies said Wednesday. There's no evidence that any of the recipients responded, officials said, preventing the hacked information from surfacing in the final months of the closely contested election. The hackers sent emails in late June and early July to people who were associated with Mr. Biden's campaign before he dropped out. The emails "contained an excerpt taken from stolen, non-public material from former President Trump's campaign as text in the emails," according to a U.S. government statement. In late July, officials with the FBI, the Office of the Director of National Intelligence and the Departmen

In [ ]:
#Extract article text from links for news sources requiring user agent
url = Newsweek_links[0]
headers = {"user-agent": "Mozilla/5.0"}
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35
article_text = scrape_article_text_useragent(url, headers)
print(article_text)

While New York is seen as a solid blue state, the Republican party still enjoys widespread support in its rural and suburban areas, and this year the gap between the GOP and the Democrat Party could be narrowing. In the 2024 presidential election, Donald Trump has the potential to gain the largest share of the vote in the Empire State than any other Republican candidate since 1988, some polls have suggested. Trump's very unlikely to win the state and get its 28 electoral votes, but a good performance in the Democratic stronghold gives Republicans a new message to bring to voters and enthusiasm about Trump could help the GOP in down ballot races. Before his rally on Long Island Wednesday night, Trump told supporters in a Manhattan bar that he was going to "win New York!" – a message he repeated to the crowd later on. "When I told some people in Washington that I'm going up to New York, we're doing a campaign speech, they said, 'What do you mean New York? You can't ever ... Republicans c

In [ ]:
#List of RSS feeds

#Fox News: Politics
foxPolRSS = f"https://moxie.foxnews.com/google-publisher/politics.xml"
#CNN News: Politics
CNNPolRSS = f"http://rss.cnn.com/rss/cnn_allpolitics.rss"
#NYT News: Politics
NYTPolRSS = f"https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml"
#https://github.com/susannapaoli/web-scraper-nyt
#NY Post: Politics
NYPostPolRSS = f"https://nypost.com/politics/feed/"
#Daily Mail: U.S.
DailyMailUSRSS = f"https://www.dailymail.co.uk/ushome/index.rss"
#NBC News: Politics
NBCPolRSS = f"https://feeds.nbcnews.com/nbcnews/public/politics"
#CNBC: Politics
CNBCPolRSS = f"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=10000113"
#CBS News: Politics
CBSPolRSS = f"https://www.cbsnews.com/latest/rss/politics"
#ABC News: Politics
ABCPolRSS = f"https://abcnews.go.com/abcnews/politicsheadlines"

In [ ]:
# @title [likly discontinuted]


# Following URL is the URL of the LLM being utilized from HuggingFace
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-xxl"
headers = {"Authorization": f"Bearer {api_token_hug}"}

# Instructions sent to the LLM in front of article text
system_input = "Analyze the text content and assign a label from {left, right, center, uncertain}. In this context, “left” indicates a left-leaning article, “right” signifies a right-leaning article, “center” implies no obvious political leaning, and “uncertain” denotes that the political orientation could not be determined. Please provide your analysis and output a new single line containing only the assigned label."

#reduce by 10% until it fits query
reduction_fraction = 0.10



result = reduce_and_query(article_text, system_input, reduction_fraction, headers)
print(result)

second_article = "https://www.cnn.com/2024/03/19/politics/texas-immigration-law-blocked-appeals/index.html"
second_result = reduce_and_query(second_article, system_input, reduction_fraction, headers)
print(second_result)

third_article = "https://www.cnn.com/videos/world/2024/03/20/israel-gaza-west-bank-settler-movement-clarissa-ward-pkg-intl-ldn-vpx.cnn"
third_result = reduce_and_query(third_article, system_input, reduction_fraction, headers)
print(third_result)

fourth_article = "https://www.cnn.com/2024/03/19/politics/trump-bond-deadline-panic/index.html"
fourth_result = reduce_and_query(fourth_article, system_input, reduction_fraction, headers)
print(fourth_result)

fifth_article = "https://www.foxnews.com/live-news/joe-biden-gop-impeachment-inquiry-hearing-hunter-biden-business-dealings"
fifth_result = reduce_and_query(fifth_article, system_input, reduction_fraction, headers)
print(fifth_result)

sixth_article = "https://www.msnbc.com/deadline-white-house/deadline-legal-blog/trump-supreme-court-immunity-appeal-delay-rcna144155"
sixth_result = reduce_and_query(sixth_article, system_input, reduction_fraction, headers)
print(sixth_result)

seventh_article = "https://www.msnbc.com/rachel-maddow-show/maddowblog/biden-white-house-reason-celebrate-falling-crime-rates-rcna144215"
seventh_result = reduce_and_query(seventh_article, system_input, reduction_fraction, headers)

[{'generated_text': 'left'}]
[{'generated_text': 'uncertain'}]
[{'generated_text': 'center'}]
[{'generated_text': 'center'}]
[{'generated_text': 'center'}]
[{'generated_text': 'center'}]
